Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [45]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division

import numpy as np
import tensorflow as tf

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
from tensorflow.contrib.learn.python import SKCompat

tf.logging.set_verbosity(tf.logging.INFO)

from six.moves import cPickle as pickle
from six.moves import range

In [46]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [47]:
t = test_labels[:10]
a = np.arange(10)
depth = 16
num_hidden = 16
#print(a)
#print(t)
#print(t[:, None])
#print(train_dataset.shape)
#print(train_dataset[0][0])
t2 = train_dataset[:2]
#print(t2.shape)
t3 = t2.reshape((-1, 28, 28, 1))
#print(t3.shape)
image_size = 28
print([image_size // 4 * image_size // 4 * depth, num_hidden])
#b = (a == t[:, None]).astype(np.float32)
#print(b)
#print(train_dataset.shape)
#c = np.zeros((10, 2))
#print(c)
#d = c.reshape((-1, 10, 2))
#print(d.shape)
#print(d)

[784, 16]


In [48]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [49]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [50]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.813728
Minibatch accuracy: 12.5%
Validation accuracy: 12.2%
Minibatch loss at step 50: 1.808777
Minibatch accuracy: 12.5%
Validation accuracy: 50.9%
Minibatch loss at step 100: 1.122523
Minibatch accuracy: 62.5%
Validation accuracy: 69.6%
Minibatch loss at step 150: 0.369430
Minibatch accuracy: 81.2%
Validation accuracy: 74.1%
Minibatch loss at step 200: 0.805793
Minibatch accuracy: 75.0%
Validation accuracy: 78.0%
Minibatch loss at step 250: 1.258946
Minibatch accuracy: 68.8%
Validation accuracy: 78.2%
Minibatch loss at step 300: 0.407371
Minibatch accuracy: 87.5%
Validation accuracy: 79.8%
Minibatch loss at step 350: 0.421301
Minibatch accuracy: 93.8%
Validation accuracy: 79.0%
Minibatch loss at step 400: 0.351300
Minibatch accuracy: 87.5%
Validation accuracy: 80.4%
Minibatch loss at step 450: 0.763328
Minibatch accuracy: 87.5%
Validation accuracy: 79.9%
Minibatch loss at step 500: 0.721722
Minibatch accuracy: 87.5%
Validation accuracy: 81.3%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [114]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')    
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    max_pool1 = tf.nn.max_pool(hidden1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv2 = tf.nn.conv2d(max_pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')    
    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    max_pool2 = tf.nn.max_pool(conv2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = max_pool2.get_shape().as_list()
    reshape = tf.reshape(max_pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden3, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [115]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.380093
Minibatch accuracy: 12.5%
Validation accuracy: 11.7%
Minibatch loss at step 50: 0.829961
Minibatch accuracy: 81.2%
Validation accuracy: 73.0%
Minibatch loss at step 100: 0.656380
Minibatch accuracy: 81.2%
Validation accuracy: 78.2%
Minibatch loss at step 150: 0.340694
Minibatch accuracy: 87.5%
Validation accuracy: 76.8%
Minibatch loss at step 200: 0.833252
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%
Minibatch loss at step 250: 0.988732
Minibatch accuracy: 75.0%
Validation accuracy: 80.5%
Minibatch loss at step 300: 0.281218
Minibatch accuracy: 93.8%
Validation accuracy: 81.9%
Minibatch loss at step 350: 0.389162
Minibatch accuracy: 93.8%
Validation accuracy: 80.2%
Minibatch loss at step 400: 0.196215
Minibatch accuracy: 93.8%
Validation accuracy: 82.3%
Minibatch loss at step 450: 0.974135
Minibatch accuracy: 75.0%
Validation accuracy: 80.9%
Minibatch loss at step 500: 0.671064
Minibatch accuracy: 87.5%
Validation accuracy: 82.9%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [64]:
# Using TF layers: tensorflow high-level API


pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat_dataset(dataset):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  return dataset
train_dataset = reformat_dataset(train_dataset)
valid_dataset = reformat_dataset(valid_dataset)
test_dataset = reformat_dataset(test_dataset)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

batch_size = 128
patch_size = 5

graph = tf.Graph()

def cnn_model(features, labels, mode):    
    """
        CNN Architecture:
            * Convolutional Layer #1: Applies 32 5x5 filters 
              (extracting 5x5-pixel subregions), with ReLU activation function
            * Pooling Layer #1: Performs max pooling with a 2x2 filter
              and stride of 2 (which specifies that pooled regions do not overlap)
            * Convolutional Layer #2: Applies 64 5x5 filters, with ReLU activation function
            * Pooling Layer #2: Again, performs max pooling with a 2x2 filter and stride of 2
            * Dense Layer #1: 1,024 neurons, with dropout regularization rate of 0.4 
              (probability of 0.4 that any given element will be dropped during training)
            * Dense Layer #2 (Logits Layer): 10 neurons, one for each digit target class (0–9).
    """
    # Input Layer
    input_layer = tf.reshape(features, [-1, image_size, image_size, 1])
    # input shape: [batch_size, 28, 28, 1]
    
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(inputs=features, 
                             filters=32, 
                             kernel_size=[patch_size, patch_size],
                             padding="same",
                             activation=tf.nn.relu)
    
    # conv1 shape: [batch_size, 28, 28, 32]

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    # pool1 shape: [batch_size, 14, 14, 1]
    
    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(inputs=pool1,
                             filters=64,
                             kernel_size=[patch_size, patch_size],
                             padding="same",
                             activation=tf.nn.relu)
    
    # conv2 shape: [batch_size, 14, 14, 64]
    
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    # pool2 shape: [batch_size, 7, 7, 1]
    
    # Dense layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == learn.ModeKeys.TRAIN)
    
    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=num_labels)
    
    loss = None
    train_op = None
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:        
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=num_labels)        

        loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    
    # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),                                       
            learning_rate=0.05,
            optimizer='SGD')
    
    # Generate Predictions
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    # Return a ModelFnOps object
    return model_fn_lib.ModelFnOps(mode=mode, predictions=predictions, loss=loss, train_op=train_op)

    

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)
Training set (200000, 28, 28, 1) (200000,)
Validation set (10000, 28, 28, 1) (10000,)
Test set (10000, 28, 28, 1) (10000,)


In [65]:
classifier = SKCompat(learn.Estimator(model_fn=cnn_model, model_dir="/tmp/convnet_model"))

# Logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

classifier.fit(
    x=train_dataset,
    y=train_labels,
    batch_size=16,
    steps=200,
    monitors=None)

# Configure the accuracy metric for evaluation
metrics = {
    "accuracy":
        learn.MetricSpec(
            metric_fn=tf.metrics.accuracy, prediction_key="classes"),
}

# Evaluate the model and print results
eval_results = classifier.score(
    x=test_dataset, y=test_labels, metrics=metrics)
print(eval_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x17abf4450>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 3202 into /tmp/convnet_model/model.ckpt.
INFO:tensorflow:loss = 1.0483, step = 3202
INFO:tensorflow:global_step/sec: 12.6766
INFO:tensorflow:loss = 1.36379, step = 3302
INFO:tensorflow:Saving checkpoints for 3401 into /tmp/convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.342576.
INFO:tensorflow:Starting evaluation at 2017-05-07-22:37:21
INFO

LeNet implementation using Tensorflow low level API

In [27]:
batch_size = 128
patch_size = 5
num_channels = 1
image_size = 28
dropout = 0.5
graph2 = tf.Graph()

with graph2.as_default():

    # Input data
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=[batch_size, image_size, image_size, num_channels])
    tf_train_labels = tf.placeholder(tf.float32, shape=[batch_size, num_labels])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    keep_prob = tf.placeholder(tf.float32)

    # Variables
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, 20], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([20]))

    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, 20, 50], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[50]))

    layer3_weights = tf.Variable(tf.truncated_normal(
        [7 * 7 * 50, 500], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[500]))

    layer4_weights = tf.Variable(tf.truncated_normal(
        [500, 10], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[10]))
                                 
    def model(data, training=False):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer1_biases)
        max_pool1 = tf.nn.max_pool(hidden1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')

        conv2 = tf.nn.conv2d(max_pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')

        hidden2 = tf.nn.relu(conv2 + layer2_biases)
        max_pool2 = tf.nn.max_pool(hidden2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')

        shape = max_pool2.get_shape().as_list()
        reshape = tf.reshape(max_pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        if training:
            keep_prob = tf.constant(dropout)
        else:
            keep_prob = tf.constant(1.0)
            
        hidden3_dropout = tf.nn.dropout(hidden3, keep_prob)
        return tf.matmul(hidden3_dropout, layer4_weights) + layer4_biases

    logits = model(tf_train_dataset, training=True)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
    decay_steps = train_labels.shape[0]
    decay_rate = 0.95
    base_learning_rate = 0.01
    learning_rate = tf.train.exponential_decay(
        base_learning_rate, 
        global_step * batch_size, 
        decay_steps, 
        decay_rate,
        staircase=True
    )
    
    #optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=global_step)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    optimizer = tf.train.AdagradOptimizer(0.05).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [28]:
num_steps = 3001

with tf.Session(graph=graph2) as session:
    tf.global_variables_initializer().run()
    for step in xrange(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, keep_prob: dropout}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' %accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Minibatch loss at step 0: 13.756258
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.410290
Minibatch accuracy: 88.3%
Validation accuracy: 86.4%
Minibatch loss at step 1000: 0.562436
Minibatch accuracy: 81.2%
Validation accuracy: 87.1%
Minibatch loss at step 1500: 0.304137
Minibatch accuracy: 93.0%
Validation accuracy: 87.8%
Minibatch loss at step 2000: 0.292231
Minibatch accuracy: 92.2%
Validation accuracy: 88.6%
Minibatch loss at step 2500: 0.331524
Minibatch accuracy: 88.3%
Validation accuracy: 88.7%
Minibatch loss at step 3000: 0.397565
Minibatch accuracy: 90.6%
Validation accuracy: 89.2%
Test accuracy: 95.0%
